## 0. Introduction

The purpose of this notebook is to explore the Black-Scholes-Merton option pricing formula with reference to chapter 4 from *Stochastic Calculus for Finance II Continuous-Time Models* (Shreve, 2008) and [this GitHub page](https://github.com/cantaro86/Financial-Models-Numerical-Methods/blob/master/1.1%20Black-Scholes%20numerical%20methods.ipynb) (Cantarutti, 2019).

## 1. Black-Scholes Closed Formula

Consider a European call option that pays $(S(T) - K)^+$ at time $T$. The strike price $K$ is some nonnegative constant. Black, Scholes, and Merton argued that the value of this call at any time should depend on the time (more precisely, on the time to expiration) and on the value of the stock price at that time, and of course it should also depend on a constant rate of interest, $r$, the volatility of the stock, $\sigma$, and the contractual strike price, $K$. Only two of these quantities, time and stock price, are variable. Following this reasoning, we let $c(t, x)$ denote the value of the call at time $t$ if the stock price at that time is $S(t) = x$. 

There is nothing random about the function $c(t, x)$. However, the value of the option is random; it is the stochastic process $c(t, S(t))$ obtained by replacing the dummy variable $x$ by the random stock price $S(t)$ in this function. At the initial time, we do not know the future stock prices $S(t)$ and hence do not know the future option values $c(t, S(t))$. Our goal is to determine the function $c(t, x)$ so we at least have a formula for the future option values in terms of the future stock prices.

Using the Itô-Doeblin formula, we can derive the Black-Scholes-Merton partial differential equation. Let the stock price $S(t)$ be a geometric Brownian motion:

$$ dS(t) = \alpha S(t) \, dt + \sigma S(t) \, dW(t). $$

Let $c(t, S(t))$ be the price at time $t \in [0, T]$ of a European call paying $(S(T) - K)^+$ at expiration time $T$. Suppose we sell this call for $X(0) = c(0, S(0))$ at time zero and, starting with initial capital $X(0)$, invest in a stock and a money market account paying a constant rate of interest $r$. If $\Delta(t)$ is the number of shares of stock held by the portfolio at time $t$, then

$$ dX(t) = \Delta(t) \, dS(t) + r(X(t) - \Delta(t) S(t)) \, dt. $$

We compute the differential of the discounted portfolio value $e^{-rt} X(t)$, the differential of the discounted call price $e^{-rt} c(t, S(t))$, and set these two equal. This results in the *delta-hedging rule*,

$$ \Delta(t) = c_x(t, S(t)), \tag{1.1} $$

and the Black-Scholes-Merton partial differential equation, 

$$ c_t(t, x) + rx c_x(t, x) + \frac{1}{2} \sigma^2 x^2 c_{xx}(t, x) = r c(t, x). $$

In addition to satisfying this partial differential equation, the funcion $c(t, x)$ must satisfy the boundary conditions

$$ c(T,x) = (x - K)^+, \quad c(t,0) = 0, \quad \lim_{x \to \infty} \left[ c(t,x) - (x - e^{-r(T-t)} K) \right] = 0. $$

The function satisfying these conditions is 

$$ c(t,x) = x N(d_1) - K e^{-r(T-t)} N(d_2), \tag{1.2} $$

with 

$$ d_1 = \frac{\ln \left( \frac{x}{K} \right) + \left(r + \frac{\sigma^2}{2} \right) (T-t)}{\sigma \sqrt{T-t}} \quad \text{and} \quad d_2 = d_1 - \sigma \sqrt{T-t}, $$

where $N$ is the cumulative distribution function of a standard normal random variable. We can interpret $N(d_1)$ and $N(d_2)$ as the risk neutral probabilities of $S(T) > K$ in the stock and money market numeraires respectively.

Using the function given by equation (1.2), if one starts with initial capital $X(0) = c(0, S(0))$ and uses the delta-hedging rule (1.1), then at every time $t$, $X(t) = c(t, S(t))$. In particular, at the final time, the value of the hedging portfolio is $X(T) = c(T, S(T)) = (S(T) - K)^+$ almost surely. The short position in the European call has been hedged.

In [1]:
import numpy as np
import scipy.stats as ss

In [2]:
# Define parameters
S0 = 100.0     # spot price
K = 100.0      # strike price
T = 1.0        # time to expiration (in years)
r = 0.04       # annual risk free rate
sigma = 0.2    # annual volatility rate

In [3]:
def closed_formula(S0, K, T, r, sigma, payoff='call'):
    """
    Calculate the price of a European option using the Black-Scholes closed formula.

    Args:
        S0 (float): Initial price (current price).
        K (float): Strike price of the option.
        T (float): Time to expiration (in years).
        r (float): Annual risk-free interest rate.
        sigma (float): Annual volatility (standard deviation of the returns).
        payoff (Literal['call', 'put']): Option payoff ('call' or 'put'). Defaults to 'call'.

    Returns:
        float: The estimated price of the option using the Black-Scholes closed formula.
    """
    d1 = (np.log(S0 / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if payoff == 'call':
        return S0 * ss.norm.cdf(d1) - K * np.exp(-r * T) * ss.norm.cdf(d2)
    elif payoff == 'put':
        return K * np.exp(-r * T) * ss.norm.cdf(-d2) - S0 * ss.norm.cdf(-d1)

In [4]:
# Calculate option prices using Black Scholes closed formula
call = closed_formula(S0, K, T, r, sigma, 'call')
put = closed_formula(S0, K, T, r, sigma, 'put')

# Print results
print(f"Call price: {call:.3f}")
print(f"Put price: {put:.3f}")

Call price: 9.925
Put price: 6.004


### 1.3 Put-Call Parity
Put–call parity defines a relationship between the price of a European call option and European put option, both with the identical strike price and expiry, namely that a portfolio of a long call option and a short put option is equivalent to (and hence has the same value as) a single forward contract at this strike price and expiry. 

This is because if the price at expiry is above the strike price, the call will be exercised, while if it is below, the put will be exercised, and thus in either case one unit of the asset will be purchased for the strike price, exactly as in a forward contract. Thus,

$$ C(t) - P(t) = S(t) - K e^{-r(T-t)}. $$

In [5]:
# Check put-call parity holds
put_call_parity = put + S0 - K * np.exp(-r * T)

# Print results
print(f"Call price: {call:.3f}")
print(f"Put-call parity price: {put_call_parity:.3f}")

Call price: 9.925
Put-call parity price: 9.925


## 2. Monte Carlo Method

Let's now simulate the random variables:

$$ S_T^i = S_0 \, \exp\left\{\left(r - \frac{\sigma^2}{2}\right)T + \sigma W_{T}^i\right\}. $$

for $1 \leq i \leq M$. 
   
The Monte Carlo estimate for the price of a European option is based on the principle that the expected payoff under the risk-neutral measure $\mathbb{Q}$, denoted $\mathbb{E}^{\mathbb{Q}} \left[ (S_T - K)^+ \right]$ for a call option (or $\mathbb{E}^{\mathbb{Q}} \left[ (K - S_T)^+ \right]$ for a put option), can be approximated by averaging the discounted payoffs across all simulations:

$$ \mathbb{E}^{\mathbb{Q}}\left[ (S_T - K)^+ \bigg| S_0 \right] \; \approx \; \frac{1}{N} \sum_{i=1}^N (S_T^i - K)^+. $$

In [6]:
def monte_carlo_method(S0, K, T, r, sigma, M, payoff='call', seed=None):
    """
    Calculate the price of a European option using Monte Carlo simulation.
    
    Args:
        S0 (float): Initial price (current price).
        K (float): Strike price of the option.
        T (float): Time to expiration (in years).
        r (float): Annual risk-free interest rate.
        sigma (float): Annual volatility (standard deviation of the returns).
        M (int): Number of Monte Carlo simulations.
        payoff (Literal['call', 'put']): Option payoff ('call' or 'put'). Defaults to 'call'.
        seed (Union[int, None]): Random seed for reproducibility. Defaults to None.
        
    Returns:
        tuple containing:
            - V (float): The estimated price of the option using Monte Carlo simulation.
            - std_err (float): The standard error of the option price using Monte Carlo simulation.
    """ 
    # Generate random normal variables for Brownian motion
    W = ss.norm.rvs(loc=(r-sigma**2/2)*T, scale=sigma*np.sqrt(T), size=M, random_state=seed)
    
    # Calculate terminal prices
    ST = S0 * np.exp(W)
    
    # Calculate discounted payoffs for call and put options
    if payoff == 'call':
        payoffs = np.exp(-r * T) * np.maximum(ST - K, 0)
        std_err = ss.sem(np.exp(-r * T) * np.maximum(ST - K, 0))
    elif payoff == 'put':
        payoffs = np.exp(-r * T) * np.maximum(K - ST, 0)
        std_err = ss.sem(np.exp(-r * T) * np.maximum(K - ST, 0))
    else:
        raise ValueError("Invalid payoff type. Specify 'call' or 'put'!")
    
    V = np.mean(payoffs)
    
    return V, std_err

In [7]:
# Calculate option prices using the Monte Carlo method
call_mc, call_mc_err = monte_carlo_method(S0, K, T, r, sigma, M=10000000, payoff='call', seed=42)
put_mc, put_mc_err = monte_carlo_method(S0, K, T, r, sigma, M=10000000, payoff='put', seed=42)

# Print results
print(f"Monte Carlo Call price: {call_mc:.3f}, with error: {call_mc_err:.6f}")
print(f"Monte Carlo Put price: {put_mc:.3f}, with error: {put_mc_err:.6f}")

Monte Carlo Call price: 9.925, with error: 0.004559
Monte Carlo Put price: 6.005, with error: 0.002847


## 3. Binomial Method

The binomial option pricing model is a discrete-time model used to price European-style options by modeling the price evolution of the underlying asset over time. 

At each time step $\Delta t$, the underlying asset price can move to either an upward or a downward price level:

$$ u = e^{\sigma \sqrt{\Delta t}}, \quad d = \frac{1}{u}. $$

The model assumes a risk-neutral probability $p$ for an up movement and $1-p$ for a down movement, such that the expected return on the underlying asset equals the risk-free rate, $r$:

$$ p = \frac{e^{r \Delta t} - d}{u - d}. $$

To price the option, a binomial tree is constructed starting from the initial stock price $S_0$. The tree evolves over $N$ time steps, where $\Delta t = \frac{T}{N}$​ is the time increment. At each step, the stock price can move up to $u S_0$ or down to $d S_0$. Under the risk-neutral measure, the expected discounted payoff of the option equals its current price under the risk-free rate, $r$.

In [8]:
def binomial_method(S0, K, T, r, sigma, N, payoff='call'):
    """
    Calculate the price of a European option using the binomial option pricing model.
    
    Args:
        S0 (float): Initial price (current price).
        K (float): Strike price of the option.
        T (float): Time to expiration (in years).
        r (float): Annual risk-free interest rate.
        sigma (float): Annual volatility (standard deviation of the returns).
        N (int): Number of time steps in the binomial model.
        payoff (Literal['call', 'put']): Option payoff ('call' or 'put'). Defaults to 'call'.
        
    Returns:
        float: The estimated price of the option using the binomial model.
    """
    dt = T / N                         # time step increment
    u = np.exp(sigma * np.sqrt(dt))    # up factor
    d = 1.0 / u                        # down factor
    
    # Initialize price vector and calculate terminal price array
    V = np.zeros(N + 1)
    ST = np.array([S0 * u**j * d**(N - j) for j in range(N + 1)])
    
    # Risk-neutral probabilities
    p = (np.exp(r * dt) - d) / (u - d)
    q = 1.0 - p
    
    # Determine option payoff type
    if payoff == 'call':
        V[:] = np.maximum(ST - K, 0.0)
    elif payoff == 'put':
        V[:] = np.maximum(K - ST, 0.0)
    else:
        raise ValueError("Invalid payoff type. Specify 'call' or 'put'!")
    
    # Backward iteration through the tree
    for i in range(N - 1, -1, -1):
        V[:-1] = np.exp(-r * dt) * (p * V[1:] + q * V[:-1])
    
    # Return the calculated option price
    return V[0]

In [9]:
# Calculate option prices using the Binomial method
call_bn = binomial_method(S0, K, T, r, sigma, N=15000, payoff='call')
put_bn = binomial_method(S0, K, T, r, sigma, N=15000, payoff='put')

# Print results
print(f"Binomial Call Price: {call_bn:.3f}")
print(f"Binomial Put Price: {put_bn:.3f}")

Binomial Call Price: 9.925
Binomial Put Price: 6.004
